In [ ]:
!pip install torch torchvision --upgrade

!pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install opencv-python matplotlib


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from google.colab import files

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(
    "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
))

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(
    "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
)

predictor = DefaultPredictor(cfg)

uploaded = files.upload()
image_path = list(uploaded)[0]

image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.imshow(image)
plt.axis("off")
plt.show()

outputs = predictor(image)

masks = outputs["instances"].pred_masks.cpu().numpy()
classes = outputs["instances"].pred_classes.cpu().numpy()

blurred_image = image.copy()

for mask,cls in zip(masks,classes):
  if cls==0:
    temp_blur = cv2.GaussianBlur(blurred_image,(21, 21), 10)
    blurred_image[mask]=temp_blur[mask]

plt.imshow(blurred_image)
plt.axis("off")
plt.show()

cv2.imwrite("blurred_output.jpg", cv2.cvtColor(blurred_image, cv2.COLOR_RGB2BGR))

